In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 6.0)
import tensorflow as tf
import importlib
from pycocotools.coco import COCO
import skimage.io as io

In [2]:
from config import ms_coco_dir
fc6_train = np.load(ms_coco_dir + '/descriptors/fc6/train.npy', encoding='latin1').item()
fc6_val = np.load(ms_coco_dir + '/descriptors/fc6/val.npy', encoding='latin1').item()
fc6_test = np.load(ms_coco_dir + '/descriptors/fc6/test.npy', encoding='latin1').item()

In [3]:
fc6 = dict(fc6_train)
fc6.update(fc6_val)
fc6.update(fc6_test)

In [4]:
del fc6_train
del fc6_val
del fc6_test

In [5]:
descriptors = fc6

In [6]:
annFileTrain = '{}/annotations/captions_train2017.json'.format(ms_coco_dir)
annFileVal = '{}/annotations/captions_val2017.json'.format(ms_coco_dir)
cocoTrain = COCO(annFileTrain)
cocoVal = COCO(annFileVal)

loading annotations into memory...
Done (t=2.92s)
creating index...
index created!
loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [7]:
trainIds = [i for i in cocoTrain.getImgIds() if i in descriptors]
valIds   = [i for i in cocoVal.getImgIds() if i in descriptors]

In [51]:
import time


def eucleidean_dist(x, y):
    z = x - y
    return np.dot(z, z)    

def cosine_dist(x, y):
    return np.dot(x,y)**2 / np.dot(x,x) / np.dot(y,y)

def knn(x_id):
    if x_id not in valIds:
        return None
    min_d = np.infty
    min_id = None
    for imgId in trainIds:
        d = eucleidean_dist(descriptors[imgId], descriptors[x_id])
        if d < min_d:
            min_d = d
            min_id = imgId
    return min_id    

def predict_val():
    predictions = {}
    imgIds = cocoVal.getImgIds()
    t_start = time.clock()
    n = len(imgIds)
    for i, x_id in enumerate(imgIds):
        print('{}/{}'.format(i+1, n))
        y_id = knn(x_id) ######
        if y_id is None:
            continue
        annIds = cocoTrain.getAnnIds(imgIds=[y_id])
        anns = cocoTrain.loadAnns(annIds)[0]
        predictions[x_id] = anns['caption']
        
        t_now = time.clock()
        t_elapsed = t_now - t_start
        str_elapsed = time.strftime('%Hh %Mm %Ss', time.gmtime(t_elapsed))
        t_eta = t_elapsed / (i+1) * (n-i-1)
        str_eta = time.strftime('%Hh %Mm %Ss', time.gmtime(t_eta))
        print('elapsed:', str_elapsed, '. ETA:', str_eta)
    return predictions

In [52]:
# %load_ext line_profiler
# %lprun -f knn predict_val()
predictions = predict_val()

1/5000
elapsed: 00h 00m 01s . ETA: 01h 39m 13s
2/5000
elapsed: 00h 00m 02s . ETA: 01h 32m 37s
3/5000
elapsed: 00h 00m 03s . ETA: 01h 30m 34s
4/5000
elapsed: 00h 00m 04s . ETA: 01h 29m 31s
5/5000
elapsed: 00h 00m 05s . ETA: 01h 29m 15s
6/5000
elapsed: 00h 00m 06s . ETA: 01h 28m 57s
7/5000
elapsed: 00h 00m 07s . ETA: 01h 28m 32s
8/5000
elapsed: 00h 00m 08s . ETA: 01h 27m 59s
9/5000
elapsed: 00h 00m 09s . ETA: 01h 28m 47s
10/5000
elapsed: 00h 00m 10s . ETA: 01h 28m 21s
11/5000
elapsed: 00h 00m 11s . ETA: 01h 28m 03s
12/5000
elapsed: 00h 00m 12s . ETA: 01h 27m 45s
13/5000
elapsed: 00h 00m 13s . ETA: 01h 27m 38s
14/5000
elapsed: 00h 00m 14s . ETA: 01h 27m 21s
15/5000
elapsed: 00h 00m 15s . ETA: 01h 27m 12s
16/5000
elapsed: 00h 00m 16s . ETA: 01h 26m 58s
17/5000
elapsed: 00h 00m 17s . ETA: 01h 26m 50s
18/5000
elapsed: 00h 00m 18s . ETA: 01h 26m 44s
19/5000
elapsed: 00h 00m 19s . ETA: 01h 26m 37s
20/5000
elapsed: 00h 00m 20s . ETA: 01h 26m 31s
21/5000
elapsed: 00h 00m 21s . ETA: 01h 26m 25s
2

elapsed: 00h 02m 55s . ETA: 01h 21m 29s
174/5000
elapsed: 00h 02m 56s . ETA: 01h 21m 28s
175/5000
elapsed: 00h 02m 57s . ETA: 01h 21m 27s
176/5000
elapsed: 00h 02m 58s . ETA: 01h 21m 26s
177/5000
elapsed: 00h 02m 59s . ETA: 01h 21m 25s
178/5000
elapsed: 00h 03m 00s . ETA: 01h 21m 24s
179/5000
elapsed: 00h 03m 01s . ETA: 01h 21m 23s
180/5000
elapsed: 00h 03m 02s . ETA: 01h 21m 22s
181/5000
elapsed: 00h 03m 03s . ETA: 01h 21m 21s
182/5000
elapsed: 00h 03m 04s . ETA: 01h 21m 20s
183/5000
elapsed: 00h 03m 05s . ETA: 01h 21m 20s
184/5000
elapsed: 00h 03m 06s . ETA: 01h 21m 19s
185/5000
elapsed: 00h 03m 07s . ETA: 01h 21m 18s
186/5000
elapsed: 00h 03m 08s . ETA: 01h 21m 17s
187/5000
elapsed: 00h 03m 09s . ETA: 01h 21m 16s
188/5000
elapsed: 00h 03m 10s . ETA: 01h 21m 15s
189/5000
elapsed: 00h 03m 11s . ETA: 01h 21m 14s
190/5000
elapsed: 00h 03m 12s . ETA: 01h 21m 17s
191/5000
elapsed: 00h 03m 13s . ETA: 01h 21m 16s
192/5000
elapsed: 00h 03m 14s . ETA: 01h 21m 16s
193/5000
elapsed: 00h 03m 15s

elapsed: 00h 05m 45s . ETA: 01h 18m 21s
343/5000
elapsed: 00h 05m 46s . ETA: 01h 18m 20s
344/5000
elapsed: 00h 05m 47s . ETA: 01h 18m 19s
345/5000
elapsed: 00h 05m 48s . ETA: 01h 18m 18s
346/5000
347/5000
elapsed: 00h 05m 49s . ETA: 01h 18m 03s
348/5000
elapsed: 00h 05m 50s . ETA: 01h 18m 02s
349/5000
elapsed: 00h 05m 51s . ETA: 01h 18m 01s
350/5000
elapsed: 00h 05m 52s . ETA: 01h 18m 00s
351/5000
elapsed: 00h 05m 53s . ETA: 01h 17m 59s
352/5000
elapsed: 00h 05m 54s . ETA: 01h 17m 58s
353/5000
elapsed: 00h 05m 55s . ETA: 01h 17m 57s
354/5000
elapsed: 00h 05m 56s . ETA: 01h 17m 56s
355/5000
elapsed: 00h 05m 57s . ETA: 01h 17m 55s
356/5000
elapsed: 00h 05m 58s . ETA: 01h 17m 54s
357/5000
elapsed: 00h 05m 59s . ETA: 01h 17m 53s
358/5000
elapsed: 00h 06m 00s . ETA: 01h 17m 52s
359/5000
elapsed: 00h 06m 01s . ETA: 01h 17m 52s
360/5000
elapsed: 00h 06m 02s . ETA: 01h 17m 51s
361/5000
elapsed: 00h 06m 03s . ETA: 01h 17m 50s
362/5000
elapsed: 00h 06m 04s . ETA: 01h 17m 49s
363/5000
elapsed: 00

elapsed: 00h 08m 35s . ETA: 01h 15m 25s
512/5000
elapsed: 00h 08m 36s . ETA: 01h 15m 24s
513/5000
elapsed: 00h 08m 37s . ETA: 01h 15m 23s
514/5000
elapsed: 00h 08m 38s . ETA: 01h 15m 22s
515/5000
elapsed: 00h 08m 39s . ETA: 01h 15m 21s
516/5000
elapsed: 00h 08m 40s . ETA: 01h 15m 20s
517/5000
elapsed: 00h 08m 41s . ETA: 01h 15m 19s
518/5000
elapsed: 00h 08m 42s . ETA: 01h 15m 18s
519/5000
elapsed: 00h 08m 43s . ETA: 01h 15m 17s
520/5000
elapsed: 00h 08m 44s . ETA: 01h 15m 16s
521/5000
elapsed: 00h 08m 45s . ETA: 01h 15m 15s
522/5000
elapsed: 00h 08m 46s . ETA: 01h 15m 14s
523/5000
elapsed: 00h 08m 47s . ETA: 01h 15m 13s
524/5000
elapsed: 00h 08m 48s . ETA: 01h 15m 12s
525/5000
elapsed: 00h 08m 49s . ETA: 01h 15m 12s
526/5000
elapsed: 00h 08m 50s . ETA: 01h 15m 11s
527/5000
elapsed: 00h 08m 51s . ETA: 01h 15m 10s
528/5000
elapsed: 00h 08m 52s . ETA: 01h 15m 09s
529/5000
elapsed: 00h 08m 53s . ETA: 01h 15m 08s
530/5000
elapsed: 00h 08m 54s . ETA: 01h 15m 07s
531/5000
elapsed: 00h 08m 55s

elapsed: 00h 11m 24s . ETA: 01h 12m 16s
683/5000
elapsed: 00h 11m 25s . ETA: 01h 12m 15s
684/5000
elapsed: 00h 11m 26s . ETA: 01h 12m 14s
685/5000
elapsed: 00h 11m 27s . ETA: 01h 12m 13s
686/5000
elapsed: 00h 11m 28s . ETA: 01h 12m 12s
687/5000
elapsed: 00h 11m 30s . ETA: 01h 12m 11s
688/5000
elapsed: 00h 11m 31s . ETA: 01h 12m 10s
689/5000
elapsed: 00h 11m 32s . ETA: 01h 12m 09s
690/5000
elapsed: 00h 11m 33s . ETA: 01h 12m 09s
691/5000
elapsed: 00h 11m 34s . ETA: 01h 12m 08s
692/5000
elapsed: 00h 11m 35s . ETA: 01h 12m 07s
693/5000
elapsed: 00h 11m 36s . ETA: 01h 12m 06s
694/5000
elapsed: 00h 11m 37s . ETA: 01h 12m 05s
695/5000
elapsed: 00h 11m 38s . ETA: 01h 12m 04s
696/5000
elapsed: 00h 11m 39s . ETA: 01h 12m 03s
697/5000
elapsed: 00h 11m 40s . ETA: 01h 12m 02s
698/5000
elapsed: 00h 11m 41s . ETA: 01h 12m 01s
699/5000
elapsed: 00h 11m 42s . ETA: 01h 12m 00s
700/5000
elapsed: 00h 11m 43s . ETA: 01h 11m 59s
701/5000
elapsed: 00h 11m 44s . ETA: 01h 11m 59s
702/5000
elapsed: 00h 11m 45s

...

elapsed: 01h 12m 31s . ETA: 00h 11m 04s
4339/5000
elapsed: 01h 12m 32s . ETA: 00h 11m 03s
4340/5000
elapsed: 01h 12m 33s . ETA: 00h 11m 02s
4341/5000
elapsed: 01h 12m 34s . ETA: 00h 11m 01s
4342/5000
elapsed: 01h 12m 35s . ETA: 00h 11m 00s
4343/5000
elapsed: 01h 12m 36s . ETA: 00h 10m 59s
4344/5000
elapsed: 01h 12m 37s . ETA: 00h 10m 58s
4345/5000
elapsed: 01h 12m 38s . ETA: 00h 10m 57s
4346/5000
elapsed: 01h 12m 39s . ETA: 00h 10m 55s
4347/5000
elapsed: 01h 12m 40s . ETA: 00h 10m 54s
4348/5000
elapsed: 01h 12m 41s . ETA: 00h 10m 53s
4349/5000
elapsed: 01h 12m 42s . ETA: 00h 10m 52s
4350/5000
elapsed: 01h 12m 43s . ETA: 00h 10m 51s
4351/5000
elapsed: 01h 12m 44s . ETA: 00h 10m 50s
4352/5000
elapsed: 01h 12m 45s . ETA: 00h 10m 49s
4353/5000
elapsed: 01h 12m 46s . ETA: 00h 10m 48s
4354/5000
elapsed: 01h 12m 47s . ETA: 00h 10m 47s
4355/5000
elapsed: 01h 12m 48s . ETA: 00h 10m 46s
4356/5000
elapsed: 01h 12m 49s . ETA: 00h 10m 45s
4357/5000
elapsed: 01h 12m 50s . ETA: 00h 10m 44s
4358/5000


elapsed: 01h 15m 17s . ETA: 00h 08m 16s
4506/5000
elapsed: 01h 15m 18s . ETA: 00h 08m 15s
4507/5000
elapsed: 01h 15m 19s . ETA: 00h 08m 14s
4508/5000
elapsed: 01h 15m 20s . ETA: 00h 08m 13s
4509/5000
elapsed: 01h 15m 21s . ETA: 00h 08m 12s
4510/5000
elapsed: 01h 15m 22s . ETA: 00h 08m 11s
4511/5000
elapsed: 01h 15m 23s . ETA: 00h 08m 10s
4512/5000
elapsed: 01h 15m 24s . ETA: 00h 08m 09s
4513/5000
elapsed: 01h 15m 25s . ETA: 00h 08m 08s
4514/5000
elapsed: 01h 15m 26s . ETA: 00h 08m 07s
4515/5000
elapsed: 01h 15m 27s . ETA: 00h 08m 06s
4516/5000
elapsed: 01h 15m 28s . ETA: 00h 08m 05s
4517/5000
elapsed: 01h 15m 29s . ETA: 00h 08m 04s
4518/5000
elapsed: 01h 15m 30s . ETA: 00h 08m 03s
4519/5000
elapsed: 01h 15m 31s . ETA: 00h 08m 02s
4520/5000
elapsed: 01h 15m 32s . ETA: 00h 08m 01s
4521/5000
elapsed: 01h 15m 33s . ETA: 00h 08m 00s
4522/5000
elapsed: 01h 15m 34s . ETA: 00h 07m 59s
4523/5000
elapsed: 01h 15m 35s . ETA: 00h 07m 58s
4524/5000
elapsed: 01h 15m 36s . ETA: 00h 07m 57s
4525/5000


elapsed: 01h 18m 04s . ETA: 00h 05m 32s
4670/5000
elapsed: 01h 18m 05s . ETA: 00h 05m 31s
4671/5000
elapsed: 01h 18m 06s . ETA: 00h 05m 30s
4672/5000
elapsed: 01h 18m 07s . ETA: 00h 05m 29s
4673/5000
elapsed: 01h 18m 08s . ETA: 00h 05m 28s
4674/5000
elapsed: 01h 18m 09s . ETA: 00h 05m 27s
4675/5000
elapsed: 01h 18m 10s . ETA: 00h 05m 26s
4676/5000
elapsed: 01h 18m 11s . ETA: 00h 05m 25s
4677/5000
elapsed: 01h 18m 12s . ETA: 00h 05m 24s
4678/5000
elapsed: 01h 18m 13s . ETA: 00h 05m 23s
4679/5000
elapsed: 01h 18m 14s . ETA: 00h 05m 22s
4680/5000
elapsed: 01h 18m 15s . ETA: 00h 05m 21s
4681/5000
elapsed: 01h 18m 16s . ETA: 00h 05m 20s
4682/5000
elapsed: 01h 18m 17s . ETA: 00h 05m 19s
4683/5000
elapsed: 01h 18m 18s . ETA: 00h 05m 18s
4684/5000
elapsed: 01h 18m 19s . ETA: 00h 05m 17s
4685/5000
elapsed: 01h 18m 20s . ETA: 00h 05m 16s
4686/5000
elapsed: 01h 18m 21s . ETA: 00h 05m 15s
4687/5000
elapsed: 01h 18m 22s . ETA: 00h 05m 14s
4688/5000
elapsed: 01h 18m 23s . ETA: 00h 05m 13s
4689/5000


elapsed: 01h 20m 58s . ETA: 00h 02m 46s
4835/5000
elapsed: 01h 21m 00s . ETA: 00h 02m 45s
4836/5000
elapsed: 01h 21m 01s . ETA: 00h 02m 44s
4837/5000
elapsed: 01h 21m 02s . ETA: 00h 02m 43s
4838/5000
elapsed: 01h 21m 03s . ETA: 00h 02m 42s
4839/5000
elapsed: 01h 21m 04s . ETA: 00h 02m 41s
4840/5000
elapsed: 01h 21m 05s . ETA: 00h 02m 40s
4841/5000
elapsed: 01h 21m 06s . ETA: 00h 02m 39s
4842/5000
elapsed: 01h 21m 08s . ETA: 00h 02m 38s
4843/5000
elapsed: 01h 21m 09s . ETA: 00h 02m 37s
4844/5000
elapsed: 01h 21m 10s . ETA: 00h 02m 36s
4845/5000
elapsed: 01h 21m 11s . ETA: 00h 02m 35s
4846/5000
elapsed: 01h 21m 12s . ETA: 00h 02m 34s
4847/5000
elapsed: 01h 21m 14s . ETA: 00h 02m 33s
4848/5000
elapsed: 01h 21m 15s . ETA: 00h 02m 32s
4849/5000
elapsed: 01h 21m 16s . ETA: 00h 02m 31s
4850/5000
elapsed: 01h 21m 17s . ETA: 00h 02m 30s
4851/5000
elapsed: 01h 21m 18s . ETA: 00h 02m 29s
4852/5000
elapsed: 01h 21m 19s . ETA: 00h 02m 28s
4853/5000
elapsed: 01h 21m 20s . ETA: 00h 02m 27s
4854/5000


elapsed: 01h 24m 24s . ETA: 00h 00m 01s
5000/5000
elapsed: 01h 24m 25s . ETA: 00h 00m 00s


In [53]:
import pickle
with open('predictions-knn-euc.pic', 'wb') as f:
    pickle.dump(predictions, f)

In [54]:
import pickle
with open('predictions-knn-euc.pic', 'rb') as f:
    predictions = pickle.load(f)

In [55]:
import json
predictions_list = [{'image_id':iid, 'caption':caption} for iid, caption in predictions.items()]
with open('predictions-knn-euc.json', 'w') as f:
    json.dump(predictions_list, f)